# TALLER 1

## 1) Cargar los archivos en formato de texto

In [ ]:
#Especificar cantidad de documentos
cant=50

In [ ]:
import os
import re
from bs4 import BeautifulSoup
formato=".txt"
inicio=0

#Creamos la carpeta de las noticias si no existe
if not os.path.exists('noticias'): 
    os.makedirs('noticias')

for cont in range(1):
    archivo = open("reuters21578/reut2-00%d%s" %(cont,".sgm"), "r")
    
    
    contenido = archivo.read()
    html = BeautifulSoup(contenido, "lxml")
    entradas = html.find_all('reuters')
    #Creamos un documento por cada noticia
    
    for i,entrada in enumerate(entradas):
        inicio=inicio+1
        texto = entrada.find('text').getText()
        documento= open("noticias/noticia%d%s" %(inicio, formato), "w")
        documento= open("noticias/noticia%d%s" %(inicio, formato), "a")
        documento.write(texto)
            
    documento.close()
    print ("%d documentos generados" %(inicio))
    archivo.close()

print ("TODOS LOS DOCUMENTOS HAN SIDO CREADOS CON EXITO")

## 2) Eliminar los stopwords (preposiciones, conectores, artículos, etc.) 

In [ ]:
import os
import re
formato=".txt"
stopwords= "(\n|\t|\-|\+|\/|'s | a | about | above | after | again | against | all | am | an | and | any | are | aren't | as | at | be | because | been | before | being | below | between | both | but | by | can't | cannot | could | couldn't | did | didn't | do | does | doesn't | doing | don't | down | during | each | few | for | from | further | had | hadn't | has | hasn't | have | haven't | having | he | he'd | he'll | he's | her | here | here's | hers | herself | him | himself | his | how | how's | i | i'd | i'll | i'm | i've | if | in | into | is | isn't | it | it's | its | itself | let's | me | more | most | mustn't | my | myself | no | nor | not | of | off | on | once | only | or | other | ought | our | ours | ourselves | out | over | own | same | shan't | she | she'd | she'll | she's | should | shouldn't | so | some | such | than | that | that's | the | their | theirs | them | themselves | then | there | there's | these | they | they'd | they'll | they're | they've | this | those | through | to | tobe | too | under | until | up | very | was | wasn't | we | we'd | we'll | we'be | we're | we've | were | weren't | what | what's | when | when's | where | where's | which | while | who | who's | whom | why | why's | with | won't | would | wouldn't | you | you'd | you'll | you're | you've | your | yours | yourself | yourselves )"
simbolos = "(\\|\'|\;|\:|\?|\¿|\<|\>|\,|\*|\(|\)|\¡|\!|\$|\#|\||\%|\&|\=|\[|\])" 

#Creamos la carpeta de los archivos temporales si no existe
if not os.path.exists('noticiasTEMP'): 
    os.makedirs('noticiasTEMP')

#Leemos cada archivo, guardamos su informacion en la variable texto y procedemos a limpiarla
for cont in range(cant):
    archivo = open("noticias/noticia%d%s" %(cont+1,formato), "r")
    texto=archivo.read()
    texto=texto.lower()
    texto=' '+texto+' '
    texto= texto.replace('.','')
    texto= texto.replace('\"',' ')
    texto=re.sub(simbolos,' ',texto)
    texto=re.sub('[0-9]','',texto)
    texto=re.sub(' +',' ',texto)
    texto=re.sub("( '|' )",' ',texto)
    for a in range(5):
        texto=re.sub(stopwords,' ',texto)
    texto=re.sub("'",' ',texto)    
    documento= open("noticiasTEMP/noticiaTEMP%d%s" %(cont+1, formato), "w")
    documento= open("noticiasTEMP/noticiaTEMP%d%s" %(cont+1, formato), "a")
    documento.write(texto)
    documento.close()
    archivo.close()
    if cont%1000==0:
        print ("%d archivos temporales generados"%(cont))


print ("TODOS LOS DOCUMENTOS <<TEMPORALES>> HAN SIDO CREADOS CON EXITO")



## 3) Construir el diccionario

In [ ]:
lista = []

for cont in range(cant):
    file = open("noticiasTEMP/noticiaTEMP%d%s" %(cont+1,".txt"), "r")
    data=file.readlines()
    file.close()

    for renglon in data:
        for palabra in renglon.split(' '):
            if palabra not in lista:
                lista.append(palabra)
    if cont%1000==0:
        print (cont)
        
lista.pop(0)
print('Se han creado %d palabras en el Diccionario'%(len(lista)))

## 4) Representar cada documento como un vector (Vector Space Model)

In [ ]:
from time import time 
matriz = []
tiempo_inicial = time() 
    
for i in range(cant):
    file = open("noticiasTEMP/noticiaTEMP%d%s" %(i+1,".txt"), "r")
    data=file.readlines()
    file.close()
    matriz.append([])
    for j in range(len(lista)):
        contador=0
        for renglon in data:
            for palabra in renglon.split(' '):
                if palabra== lista[j]:
                    contador=contador+1
        matriz[i].append(contador)
    if i%10==0:
        print('%d filas generadas'%(i))        
tiempo_final = time() 
tiempo_ejecucion = tiempo_final - tiempo_inicial                
print ('Hecho: Tiempo de ejecución =%d' %(tiempo_ejecucion))

## 5) Matriz TFIDF
        

In [ ]:
import math
listacuenta=[]
for j in range(len(lista)):
    contador=0
    for i in range(cant):
        if matriz[i][j]!=0:
            contador=contador+1
    listacuenta.append(contador)

tdfidf=[]   
for i in range(cant):
    tdfidf.append([])
    for j in range (len(lista)):
        if matriz[i][j]==0:
            tdfidf[i].append(0)
        else:
            valor=matriz[i][j]*math.log((cant/(listacuenta[j])),10)
            tdfidf[i].append(valor)
print (tdfidf)            
            

## 6) Indice Invertido

In [ ]:
indiceinv=[]
for j in range(len(lista)):
    contador=0
    indiceinv.append([])
    for i in range(cant):
        if matriz[i][j]!=0:
            indiceinv[j].append(i+1)
            
print(indiceinv)

## 7) Buscador

In [ ]:
def busqueda(query):
    query=' '+query+' '
    query=query.lower()
    query= query.replace('.','')
    query= query.replace('\"',' ')
    query=re.sub(simbolos,' ',query)
    query=re.sub('[0-9]','',query)
    query=re.sub(' +',' ',query)
    query=re.sub("( '|' )",' ',query)
    for a in range(5):
        query=re.sub(stopwords,' ',query)
    return query

def diccionario_query(query):
    dicquery=query.split(" ")
    dicquery.pop(0)
    dicquery.pop(-1)
    return dicquery


def frec_query(listaquery):
    frec_query=[]*len(lista)
    for i in range(len(lista)):
        frec_query.append(0)

    for i in range(len(listaquery)):
        if listaquery[i] in lista:
            ind=lista.index(listaquery[i])
            valor = frec_query[ind]+1
            frec_query[ind]=valor
    return frec_query    
        

def tfidf_query(frec_query):
    tfidf_query=[]
    for i in range(len(frec_query)):
        tfidf_query.append(0)
    for i in range(len(frec_query)):
        if frec_query[i]!=0:
            tfidf_query[i]=frec_query[i]*math.log((cant/(listacuenta[i])),10)
           
    return tfidf_query 


def consul_invertido(frec_query,indiceinv):
    arraydocumento=[]
    for i in range(len(frec_query)):
        if frec_query[i]!=0:
            arraydocumento=list(set().union(arraydocumento, indiceinv[i]))
    return arraydocumento        
 
def producto_punto(arraydocumento, tfidf_query):
    resultados=[]
    for i in range(len(arraydocumento)):
        resultados
        c=arraydocumento[i]
        suma=0
        sumaden1=0
        sumaden2=0
        for r in range(len(lista)):
            suma=suma+(tdfidf[c-1][r]*tfidf_query[r])
            sumaden1=sumaden1+pow(tfidf_query[r],2)
            sumaden2=sumaden2+pow(tdfidf[c-1][r],2)
        
        simcos=(suma/(math.sqrt(sumaden1)+math.sqrt(sumaden2)))
        resultados.append(simcos)
    return resultados

    
    

In [ ]:
from ipywidgets import widgets
from IPython.core.display import display, HTML
from IPython.display import display
from itertools import islice
button=widgets.Button(description="Buscar")
text=widgets.Text()
display(text) 
display(button)

def mititulo(documentos, resultados):
    display(HTML('<h1>RESULTADOS</h1>'))
    for i in range(len(documentos)):
        valor=max(resultados)
        pos=resultados.index(valor)
        noticia=documentos[pos]
        file = open("noticias/noticia%d%s" %(noticia,".txt"), "r")
        tit=islice(file,1,2)
        desc=islice(file,0,4)
        descripcion=""
        for linea in tit:
            titulo=linea
        for linea in desc:
            descripcion=descripcion+" "+linea
        descripcion= descripcion.replace('\n',' ')
        file.close()
        display(HTML('<b><a href="noticias/noticia%d.txt" target="_blank">[%d] %s</a></b>%s...'%(noticia,i+1,titulo,descripcion)))
        resultados.pop(pos)
        documentos.pop(pos)

          
        

        
        
def handle_submit (sender):
    busqueda(text.value)
    


def actionboton(b):
    query=busqueda(text.value)
    lista=diccionario_query(query)
    queryfrecuencias=frec_query(lista)
    tfidfquery=tfidf_query(queryfrecuencias)
    documentos=consul_invertido(queryfrecuencias, indiceinv)
    producto=producto_punto(documentos,tfidfquery)
    mititulo(documentos, producto)
    
    
    
    
text.on_submit(handle_submit)
button.on_click(actionboton)


[3, 29, 4]
